In [2]:
import numpy as np
import matplotlib

from matplotlib import pyplot as plt
import cv2
import supervision as sv
import os

ModuleNotFoundError: No module named 'torch'

In [3]:
def pixel_accuracy(pred, target):
    correct = (pred == target).sum()
    total = pred.size
    return correct / total

def intersection_over_union(pred, target):
    intersection = (pred & target).sum()
    union = (pred | target).sum()
    return intersection / union

def dice_coefficient(pred, target):
    intersection = (pred & target).sum()
    return 2 * intersection / (pred.sum() + target.sum())



def calculate_metrics(pred_list, target_list):
    total_pixel_acc = 0
    total_iou = 0
    total_dice = 0
    n = len(pred_list)  # Assuming pred_list and target_list are of the same length

    for pred, target in zip(pred_list, target_list):
        total_pixel_acc += pixel_accuracy(pred, target)
        total_iou += intersection_over_union(pred, target)
        total_dice += dice_coefficient(pred, target)

    # Calculate the mean for each metric
    mean_pixel_acc = total_pixel_acc / n
    mean_iou = total_iou / n
    mean_dice = total_dice / n

    return mean_pixel_acc, mean_iou, mean_dice


def get_segmentation_mask_from_image(img, shape=(1920, 1080)):
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    masks = img > 0
    return masks
    


In [43]:
import os
import cv2

original_mask_folder = 'pad_segmentation_all/all-mask'
sessoes_folder = "./sessoes"


def process_all_images(sessoes_folder, original_mask_folder):


    groups_metrics_sam = {"group_1":{}, "group_2":{}, "group_3":{}, "group_4":{}} 
    groups_metrics_manual = {"group_1":{}, "group_2":{}, "group_3":{}, "group_4":{}} 
    #create a dictionary to store the metrics for each group
    #create pixel accuracy, iou, dice and lenght for each group
    
    iou_sam_sum = 0
    iou_manual_sum = 0
    dice_sam_sum = 0
    dice_manual_sum = 0
    pixel_acc_sam_sum = 0
    pixel_acc_manual_sum = 0
    n_sam = 0
    n_manual = 0 

    for sessao in os.listdir(sessoes_folder):
        #get session group that is in the sessao name 
        for group in groups_metrics_sam.keys():
            if group in sessao:
                group_name = group
                break
        #get all images in the session folder 
        for image in os.listdir(os.path.join(sessoes_folder, sessao, "masked")):
            #get same mask from the original mask folder
            original_mask_name = image.replace('.jpg', '_segmentation.png')
            original_mask_path = os.path.join(original_mask_folder, original_mask_name)
            
            if not os.path.exists(original_mask_path):
                print(f"Image {original_mask_path} not found in the original mask folder")
                continue
   
            #load images
            mask = cv2.imread(os.path.join(sessoes_folder, sessao, "masked", image))
            original_mask_img = cv2.imread(original_mask_path)
            # Resize logic
            if original_mask_img.shape[:2] != mask.shape[:2]:
                target_shape = (min(original_mask_img.shape[1], mask.shape[1]), min(original_mask_img.shape[0], mask.shape[0]))
                original_mask_img = cv2.resize(original_mask_img, target_shape[::-1])  # Reverse shape for width, height
                mask = cv2.resize(mask, target_shape[::-1])

            original_mask = get_segmentation_mask_from_image(original_mask_img, original_mask_img.shape)
            mask = get_segmentation_mask_from_image(mask, original_mask_img.shape)

            if "sam" in sessao: 
                pixel_acc, iou, dice = calculate_metrics([mask], [original_mask])
                pixel_acc_sam_sum += pixel_acc
                iou_sam_sum += iou
                dice_sam_sum += dice
                n_sam += 1
                if group_name not in groups_metrics_sam.keys():
                    groups_metrics_sam[group_name] = {"pixel_acc":pixel_acc, "iou":iou, "dice":dice, "lenght":1}
                else:
                    groups_metrics_sam[group_name]["pixel_acc"] = groups_metrics_sam[group_name].get("pixel_acc", 0) + pixel_acc
                    groups_metrics_sam[group_name]["iou"] = groups_metrics_sam[group_name].get("iou", 0) + iou
                    groups_metrics_sam[group_name]["dice"] = groups_metrics_sam[group_name].get("dice", 0) + dice
                    groups_metrics_sam[group_name]["lenght"] = groups_metrics_sam[group_name].get("lenght", 0) + 1
            else:
                pixel_acc, iou, dice = calculate_metrics([mask], [original_mask])
                pixel_acc_manual_sum += pixel_acc
                iou_manual_sum += iou
                dice_manual_sum += dice
                n_manual += 1
                if group_name not in groups_metrics_manual.keys():
                    groups_metrics_manual[group_name] = {"pixel_acc":pixel_acc, "iou":iou, "dice":dice, "lenght":1}
                else:
                    groups_metrics_manual[group_name]["pixel_acc"] =  groups_metrics_manual[group_name].get("pixel_acc", 0) + pixel_acc
                    groups_metrics_manual[group_name]["iou"] = groups_metrics_manual[group_name].get("iou", 0) + iou
                    groups_metrics_manual[group_name]["dice"] = groups_metrics_manual[group_name].get("dice", 0) + dice
                    groups_metrics_manual[group_name]["lenght"] = groups_metrics_manual[group_name].get("lenght", 0) + 1


       


    print("Mean pixel accuracy for SAM: ", pixel_acc_sam_sum  / n_sam)
    print("Mean IOU for SAM: ", iou_sam_sum / n_sam)
    print("Mean Dice for SAM: ", dice_sam_sum / n_sam)
    print("Number of images for SAM: ", n_sam)
    print("Mean pixel accuracy for Manual: ", pixel_acc_manual_sum / n_manual)
    print("Mean IOU for Manual: ", iou_manual_sum / n_manual)
    print("Mean Dice for Manual: ", dice_manual_sum / n_manual)
    print("Number of images for Manual: ", n_manual)
    
    print("Groups metrics for SAM: ")
    
    for group in groups_metrics_sam.keys():
        print(group)
        print("Mean pixel accuracy: ", groups_metrics_sam[group]["pixel_acc"] / groups_metrics_sam[group]["lenght"])
        print("Mean IOU: ", groups_metrics_sam[group]["iou"] / groups_metrics_sam[group]["lenght"])
        print("Mean Dice: ", groups_metrics_sam[group]["dice"] / groups_metrics_sam[group]["lenght"])
        print("Number of images: ", groups_metrics_sam[group]["lenght"])
    print("Groups metrics for Manual: ")
    for group in groups_metrics_manual.keys():
        print(group)
        print("Mean pixel accuracy: ", groups_metrics_manual[group]["pixel_acc"] / groups_metrics_manual[group]["lenght"])
        print("Mean IOU: ", groups_metrics_manual[group]["iou"] / groups_metrics_manual[group]["lenght"])
        print("Mean Dice: ", groups_metrics_manual[group]["dice"] / groups_metrics_manual[group]["lenght"])
        print("Number of images: ", groups_metrics_manual[group]["lenght"])
import os
import cv2

original_mask_folder = 'pad_segmentation_all/all-mask'
sessoes_folder = "./sessoes"


def process_all_images(sessoes_folder, original_mask_folder):


    groups_metrics_sam = {"group_1":{}, "group_2":{}, "group_3":{}, "group_4":{}} 
    groups_metrics_manual = {"group_1":{}, "group_2":{}, "group_3":{}, "group_4":{}} 
    #create a dictionary to store the metrics for each group
    #create pixel accuracy, iou, dice and lenght for each group
    
    iou_sam_sum = 0
    iou_manual_sum = 0
    dice_sam_sum = 0
    dice_manual_sum = 0
    pixel_acc_sam_sum = 0
    pixel_acc_manual_sum = 0
    n_sam = 0
    n_manual = 0 

    for sessao in os.listdir(sessoes_folder):
        #get session group that is in the sessao name 
        for group in groups_metrics_sam.keys():
            if group in sessao:
                group_name = group
                break
        #get all images in the session folder 
        for image in os.listdir(os.path.join(sessoes_folder, sessao, "masked")):
            #get same mask from the original mask folder
            original_mask_name = image.replace('.jpg', '_segmentation.png')
            original_mask_path = os.path.join(original_mask_folder, original_mask_name)
            
            if not os.path.exists(original_mask_path):
                print(f"Image {original_mask_path} not found in the original mask folder")
                continue
   
            #load images
            mask = cv2.imread(os.path.join(sessoes_folder, sessao, "masked", image))
            original_mask_img = cv2.imread(original_mask_path)
            # Resize logic
            if original_mask_img.shape[:2] != mask.shape[:2]:
                target_shape = (min(original_mask_img.shape[1], mask.shape[1]), min(original_mask_img.shape[0], mask.shape[0]))
                original_mask_img = cv2.resize(original_mask_img, target_shape[::-1])  # Reverse shape for width, height
                mask = cv2.resize(mask, target_shape[::-1])

            original_mask = get_segmentation_mask_from_image(original_mask_img, original_mask_img.shape)
            mask = get_segmentation_mask_from_image(mask, original_mask_img.shape)

            if "sam" in sessao: 
                pixel_acc, iou, dice = calculate_metrics([mask], [original_mask])
                pixel_acc_sam_sum += pixel_acc
                iou_sam_sum += iou
                dice_sam_sum += dice
                n_sam += 1
                if group_name not in groups_metrics_sam.keys():
                    groups_metrics_sam[group_name] = {"pixel_acc":pixel_acc, "iou":iou, "dice":dice, "lenght":1}
                else:
                    groups_metrics_sam[group_name]["pixel_acc"] = groups_metrics_sam[group_name].get("pixel_acc", 0) + pixel_acc
                    groups_metrics_sam[group_name]["iou"] = groups_metrics_sam[group_name].get("iou", 0) + iou
                    groups_metrics_sam[group_name]["dice"] = groups_metrics_sam[group_name].get("dice", 0) + dice
                    groups_metrics_sam[group_name]["lenght"] = groups_metrics_sam[group_name].get("lenght", 0) + 1
            else:
                pixel_acc, iou, dice = calculate_metrics([mask], [original_mask])
                pixel_acc_manual_sum += pixel_acc
                iou_manual_sum += iou
                dice_manual_sum += dice
                n_manual += 1
                if group_name not in groups_metrics_manual.keys():
                    groups_metrics_manual[group_name] = {"pixel_acc":pixel_acc, "iou":iou, "dice":dice, "lenght":1}
                else:
                    groups_metrics_manual[group_name]["pixel_acc"] =  groups_metrics_manual[group_name].get("pixel_acc", 0) + pixel_acc
                    groups_metrics_manual[group_name]["iou"] = groups_metrics_manual[group_name].get("iou", 0) + iou
                    groups_metrics_manual[group_name]["dice"] = groups_metrics_manual[group_name].get("dice", 0) + dice
                    groups_metrics_manual[group_name]["lenght"] = groups_metrics_manual[group_name].get("lenght", 0) + 1


       


    print("Mean pixel accuracy for SAM: ", pixel_acc_sam_sum  / n_sam)
    print("Mean IOU for SAM: ", iou_sam_sum / n_sam)
    print("Mean Dice for SAM: ", dice_sam_sum / n_sam)
    print("Number of images for SAM: ", n_sam)
    print("Mean pixel accuracy for Manual: ", pixel_acc_manual_sum / n_manual)
    print("Mean IOU for Manual: ", iou_manual_sum / n_manual)
    print("Mean Dice for Manual: ", dice_manual_sum / n_manual)
    print("Number of images for Manual: ", n_manual)
    
    print("Groups metrics for SAM: ")
    
    for group in groups_metrics_sam.keys():
        print(group)
        print("Mean pixel accuracy: ", groups_metrics_sam[group]["pixel_acc"] / groups_metrics_sam[group]["lenght"])
        print("Mean IOU: ", groups_metrics_sam[group]["iou"] / groups_metrics_sam[group]["lenght"])
        print("Mean Dice: ", groups_metrics_sam[group]["dice"] / groups_metrics_sam[group]["lenght"])
        print("Number of images: ", groups_metrics_sam[group]["lenght"])
    print("Groups metrics for Manual: ")
    for group in groups_metrics_manual.keys():
        print(group)
        print("Mean pixel accuracy: ", groups_metrics_manual[group]["pixel_acc"] / groups_metrics_manual[group]["lenght"])
        print("Mean IOU: ", groups_metrics_manual[group]["iou"] / groups_metrics_manual[group]["lenght"])
        print("Mean Dice: ", groups_metrics_manual[group]["dice"] / groups_metrics_manual[group]["lenght"])
        print("Number of images: ", groups_metrics_manual[group]["lenght"])


    #for each group, the manual + sam metrics as well
    print("Groups metrics for SAM + Manual: ")
    for group in groups_metrics_sam.keys():
        print(group)
        print("Mean pixel accuracy: ", (groups_metrics_sam[group]["pixel_acc"] + groups_metrics_manual[group]["pixel_acc"]) / (groups_metrics_sam[group]["lenght"] + groups_metrics_manual[group]["lenght"]))
        print("Mean IOU: ", (groups_metrics_sam[group]["iou"] + groups_metrics_manual[group]["iou"]) / (groups_metrics_sam[group]["lenght"] + groups_metrics_manual[group]["lenght"]))
        print("Mean Dice: ", (groups_metrics_sam[group]["dice"] + groups_metrics_manual[group]["dice"]) / (groups_metrics_sam[group]["lenght"] + groups_metrics_manual[group]["lenght"]))
        print("Number of images: ", groups_metrics_sam[group]["lenght"] + groups_metrics_manual[group]["lenght"])



process_all_images(sessoes_folder, original_mask_folder)






   






   

Mean pixel accuracy for SAM:  0.916274942212961
Mean IOU for SAM:  0.7444199407931513
Mean Dice for SAM:  0.8264605739496885
Number of images for SAM:  1200
Mean pixel accuracy for Manual:  0.9459723693247953
Mean IOU for Manual:  0.7414383376471712
Mean Dice for Manual:  0.8425491173429035
Number of images for Manual:  1200
Groups metrics for SAM: 
group_1
Mean pixel accuracy:  0.9416582910839071
Mean IOU:  0.7513436464465216
Mean Dice:  0.8312487475656634
Number of images:  250
group_2
Mean pixel accuracy:  0.8454613211685138
Mean IOU:  0.6855150036462546
Mean Dice:  0.7666699914625799
Number of images:  350
group_3
Mean pixel accuracy:  0.9379526074401622
Mean IOU:  0.7664346110989569
Mean Dice:  0.8518866042779137
Number of images:  300
group_4
Mean pixel accuracy:  0.9560603774784986
Mean IOU:  0.7853579424475803
Mean Dice:  0.8668000785097765
Number of images:  300
Groups metrics for Manual: 
group_1
Mean pixel accuracy:  0.9494489996799482
Mean IOU:  0.7179807546123992
Mean Dice

In [3]:
import os
import cv2

original_mask_folder = 'pad_segmentation_all/all-mask'
sessoes_folder = "./sessoes"



def process_all_images(sessoes_folder, original_mask_folder):


    groups_metrics_sam = {"group_1":{}, "group_2":{}, "group_3":{}, "group_4":{}} 
    groups_metrics_manual = {"group_1":{}, "group_2":{}, "group_3":{}, "group_4":{}} 
    #create a dictionary to store the metrics for each group
    #create pixel accuracy, iou, dice and lenght for each group
    
    iou_sam_sum = 0
    iou_manual_sum = 0
    dice_sam_sum = 0
    dice_manual_sum = 0
    pixel_acc_sam_sum = 0
    pixel_acc_manual_sum = 0
    n_sam = 0
    n_manual = 0 

    for sessao in os.listdir(sessoes_folder):
        #get session group that is in the sessao name 
        for group in groups_metrics_sam.keys():
            if group in sessao:
                group_name = group
                break
        #get all images in the session folder 
        for image in os.listdir(os.path.join(sessoes_folder, sessao, "masked")):
            #get same mask from the original mask folder
            original_mask_name = image.replace('.jpg', '_segmentation.png')
            original_mask_path = os.path.join(original_mask_folder, original_mask_name)
            
            if not os.path.exists(original_mask_path):
                print(f"Image {original_mask_path} not found in the original mask folder")
                continue
   
            #load images
            mask = cv2.imread(os.path.join(sessoes_folder, sessao, "masked", image))
            original_mask_img = cv2.imread(original_mask_path)
            # Resize logic
            if original_mask_img.shape[:2] != mask.shape[:2]:
                target_shape = (min(original_mask_img.shape[1], mask.shape[1]), min(original_mask_img.shape[0], mask.shape[0]))
                original_mask_img = cv2.resize(original_mask_img, target_shape[::-1])  # Reverse shape for width, height
                mask = cv2.resize(mask, target_shape[::-1])

            original_mask = get_segmentation_mask_from_image(original_mask_img, original_mask_img.shape)
            mask = get_segmentation_mask_from_image(mask, original_mask_img.shape)

            if "sam" in sessao: 
                pixel_acc, iou, dice = calculate_metrics([mask], [original_mask])
                pixel_acc_sam_sum += pixel_acc
                iou_sam_sum += iou
                dice_sam_sum += dice
                n_sam += 1
                if group_name not in groups_metrics_sam.keys():
                    groups_metrics_sam[group_name] = {"pixel_acc":pixel_acc, "iou":iou, "dice":dice, "lenght":1}
                else:
                    groups_metrics_sam[group_name]["pixel_acc"] = groups_metrics_sam[group_name].get("pixel_acc", 0) + pixel_acc
                    groups_metrics_sam[group_name]["iou"] = groups_metrics_sam[group_name].get("iou", 0) + iou
                    groups_metrics_sam[group_name]["dice"] = groups_metrics_sam[group_name].get("dice", 0) + dice
                    groups_metrics_sam[group_name]["lenght"] = groups_metrics_sam[group_name].get("lenght", 0) + 1
            else:
                pixel_acc, iou, dice = calculate_metrics([mask], [original_mask])
                pixel_acc_manual_sum += pixel_acc
                iou_manual_sum += iou
                dice_manual_sum += dice
                n_manual += 1
                if group_name not in groups_metrics_manual.keys():
                    groups_metrics_manual[group_name] = {"pixel_acc":pixel_acc, "iou":iou, "dice":dice, "lenght":1}
                else:
                    groups_metrics_manual[group_name]["pixel_acc"] =  groups_metrics_manual[group_name].get("pixel_acc", 0) + pixel_acc
                    groups_metrics_manual[group_name]["iou"] = groups_metrics_manual[group_name].get("iou", 0) + iou
                    groups_metrics_manual[group_name]["dice"] = groups_metrics_manual[group_name].get("dice", 0) + dice
                    groups_metrics_manual[group_name]["lenght"] = groups_metrics_manual[group_name].get("lenght", 0) + 1


       


    print("Mean pixel accuracy for SAM: ", pixel_acc_sam_sum  / n_sam)
    print("Mean IOU for SAM: ", iou_sam_sum / n_sam)
    print("Mean Dice for SAM: ", dice_sam_sum / n_sam)
    print("Number of images for SAM: ", n_sam)
    print("Mean pixel accuracy for Manual: ", pixel_acc_manual_sum / n_manual)
    print("Mean IOU for Manual: ", iou_manual_sum / n_manual)
    print("Mean Dice for Manual: ", dice_manual_sum / n_manual)
    print("Number of images for Manual: ", n_manual)
    
    print("Groups metrics for SAM: ")
    
    for group in groups_metrics_sam.keys():
        print(group)
        print("Mean pixel accuracy: ", groups_metrics_sam[group]["pixel_acc"] / groups_metrics_sam[group]["lenght"])
        print("Mean IOU: ", groups_metrics_sam[group]["iou"] / groups_metrics_sam[group]["lenght"])
        print("Mean Dice: ", groups_metrics_sam[group]["dice"] / groups_metrics_sam[group]["lenght"])
        print("Number of images: ", groups_metrics_sam[group]["lenght"])
    print("Groups metrics for Manual: ")
    for group in groups_metrics_manual.keys():
        print(group)
        print("Mean pixel accuracy: ", groups_metrics_manual[group]["pixel_acc"] / groups_metrics_manual[group]["lenght"])
        print("Mean IOU: ", groups_metrics_manual[group]["iou"] / groups_metrics_manual[group]["lenght"])
        print("Mean Dice: ", groups_metrics_manual[group]["dice"] / groups_metrics_manual[group]["lenght"])
        print("Number of images: ", groups_metrics_manual[group]["lenght"])

process_all_images(sessoes_folder, original_mask_folder)






   






   

Mean pixel accuracy for SAM:  0.916274942212961
Mean IOU for SAM:  0.7444199407931513
Mean Dice for SAM:  0.8264605739496885
Number of images for SAM:  1200
Mean pixel accuracy for Manual:  0.9459723693247953
Mean IOU for Manual:  0.7414383376471712
Mean Dice for Manual:  0.8425491173429035
Number of images for Manual:  1200
Groups metrics for SAM: 
group_1
Mean pixel accuracy:  0.9416582910839071
Mean IOU:  0.7513436464465216
Mean Dice:  0.8312487475656634
Number of images:  250
group_2
Mean pixel accuracy:  0.8454613211685138
Mean IOU:  0.6855150036462546
Mean Dice:  0.7666699914625799
Number of images:  350
group_3
Mean pixel accuracy:  0.9379526074401622
Mean IOU:  0.7664346110989569
Mean Dice:  0.8518866042779137
Number of images:  300
group_4
Mean pixel accuracy:  0.9560603774784986
Mean IOU:  0.7853579424475803
Mean Dice:  0.8668000785097765
Number of images:  300
Groups metrics for Manual: 
group_1
Mean pixel accuracy:  0.9494489996799482
Mean IOU:  0.7179807546123992
Mean Dice

In [12]:
import os
import cv2
import numpy as np
original_mask_folder = 'pad_segmentation_all/all-mask'
sessoes_folder = "./sessoes"



def process_all_images(sessoes_folder, original_mask_folder):


    groups_metrics_sam = {"group_1":{}, "group_2":{}, "group_3":{}, "group_4":{}} 
    groups_metrics_manual = {"group_1":{}, "group_2":{}, "group_3":{}, "group_4":{}} 
    #create a dictionary to store the metrics for each group
    #create pixel accuracy, iou, dice and lenght for each group
    
    iou_sam_sum = []
    iou_manual_sum = []
    dice_sam_sum = []
    dice_manual_sum = []
    pixel_acc_sam_sum = []
    pixel_acc_manual_sum = []
    n_sam = 0
    n_manual = 0 
    times_sam = []
    times_manual = []

    for sessao in os.listdir(sessoes_folder):
        #get session group that is in the sessao name 
        for group in groups_metrics_sam.keys():
            if group in sessao:
                group_name = group
                break
        #get all images in the session folder 
        for image in os.listdir(os.path.join(sessoes_folder, sessao, "masked")):
            #get same mask from the original mask folder
            original_mask_name = image.replace('.jpg', '_segmentation.png')
            original_mask_path = os.path.join(original_mask_folder, original_mask_name)
            
            if not os.path.exists(original_mask_path):
                print(f"Image {original_mask_path} not found in the original mask folder")
                continue
   
            #load images
            mask = cv2.imread(os.path.join(sessoes_folder, sessao, "masked", image))
            original_mask_img = cv2.imread(original_mask_path)
            # Resize logic
            if original_mask_img.shape[:2] != mask.shape[:2]:
                target_shape = (min(original_mask_img.shape[1], mask.shape[1]), min(original_mask_img.shape[0], mask.shape[0]))
                original_mask_img = cv2.resize(original_mask_img, target_shape[::-1])  # Reverse shape for width, height
                mask = cv2.resize(mask, target_shape[::-1])

            original_mask = get_segmentation_mask_from_image(original_mask_img, original_mask_img.shape)
            mask = get_segmentation_mask_from_image(mask, original_mask_img.shape)

            #read time.txt file to get the sesssion seconds time
            with open(os.path.join(sessoes_folder, sessao, "time.txt"), "r") as f:
                time = float(f.readlines()[0])
                

            if "sam" in sessao: 
                pixel_acc, iou, dice = calculate_metrics([mask], [original_mask])
                pixel_acc_sam_sum.append(pixel_acc)
                iou_sam_sum.append(iou)
                dice_sam_sum.append(dice)
                times_sam.append(time)
                n_sam += 1
                if "pixel_acc" not in groups_metrics_sam[group_name].keys():
                    groups_metrics_sam[group_name] = {"pixel_acc":[pixel_acc], "iou":[iou], "dice":[dice], "lenght":1, "time":[time]}
                else:
                    groups_metrics_sam[group_name]["pixel_acc"].append(pixel_acc)
                    groups_metrics_sam[group_name]["iou"].append(iou)
                    groups_metrics_sam[group_name]["dice"].append(dice)
                    groups_metrics_sam[group_name]["lenght"] = groups_metrics_sam[group_name].get("lenght", 0) + 1
                    groups_metrics_sam[group_name]["time"].append(time)
            else:
                pixel_acc, iou, dice = calculate_metrics([mask], [original_mask])
                pixel_acc_manual_sum.append(pixel_acc)
                iou_manual_sum.append(iou)
                dice_manual_sum.append(dice)
                times_manual.append(time)
                n_manual += 1
                if "pixel_acc" not in groups_metrics_manual[group_name].keys():
                    groups_metrics_manual[group_name] = {"pixel_acc":[pixel_acc], "iou":[iou], "dice":[dice], "lenght":1, "time":[time]}
                else:
                    groups_metrics_manual[group_name]["pixel_acc"].append(pixel_acc)
                    groups_metrics_manual[group_name]["iou"].append(iou)
                    groups_metrics_manual[group_name]["dice"].append(dice)
                    groups_metrics_manual[group_name]["lenght"] = groups_metrics_manual[group_name].get("lenght", 0) + 1
                    groups_metrics_manual[group_name]["time"].append(time)


       
    print("Mean pixel accuracy for SAM: {:.3f}".format(sum(pixel_acc_sam_sum) / n_sam),"+- {:.3f}".format(np.std(pixel_acc_sam_sum)))
    print("Mean IOU for SAM: {:.3f}".format(sum(iou_sam_sum) / n_sam),"+- {:.3f}".format(np.std(iou_sam_sum)))
    print("Mean Dice for SAM: {:.3f}".format(sum(dice_sam_sum) / n_sam),"+- {:.3f}".format(np.std(dice_sam_sum)))
    print("Number of images for SAM: ", n_sam)
    print("Mean time for SAM: {:.3f}".format(sum(times_sam) / n_sam),"+- {:.3f}".format(np.std(times_sam)))

    print()
    print("Mean pixel accuracy for Manual: {:.3f}".format(sum(pixel_acc_manual_sum) / n_manual),"+- {:.3f}".format(np.std(pixel_acc_manual_sum)))
    print("Mean IOU for Manual: {:.3f}".format(sum(iou_manual_sum) / n_manual),"+- {:.3f}".format(np.std(iou_manual_sum)))
    print("Mean Dice for Manual: {:.3f}".format(sum(dice_manual_sum) / n_manual),"+- {:.3f}".format(np.std(dice_manual_sum)))
    print("Number of images for Manual: ", n_manual)
    print("Mean time for Manual: {:.3f}".format(sum(times_manual) / n_manual),"+- {:.3f}".format(np.std(times_manual)))
    print()

    print("Mean pixel accuracu for SAM + Manual: {:.3f}".format((sum(pixel_acc_sam_sum) + sum(pixel_acc_manual_sum)) / (n_sam + n_manual)),"+- {:.3f}".format(np.std(pixel_acc_sam_sum + pixel_acc_manual_sum)))
    print("Mean IOU for SAM + Manual: {:.3f}".format((sum(iou_sam_sum) + sum(iou_manual_sum)) / (n_sam + n_manual)),"+- {:.3f}".format(np.std(iou_sam_sum + iou_manual_sum)))
    print("Mean Dice for SAM + Manual: {:.3f}".format((sum(dice_sam_sum) + sum(dice_manual_sum)) / (n_sam + n_manual)),"+- {:.3f}".format(np.std(dice_sam_sum + dice_manual_sum)))
    print("Number of images for SAM + Manual: ", n_sam + n_manual)
    print("Mean time for SAM + Manual: {:.3f}".format((sum(times_sam) + sum(times_manual)) / (n_sam + n_manual)),"+- {:.3f}".format(np.std(times_sam + times_manual)))

    print("Groups metrics for SAM: ")
    for group in groups_metrics_sam.keys():
        print("Group: ", group)
        print("Mean pixel accuracy: {:.3f}".format(sum(groups_metrics_sam[group]["pixel_acc"]) / groups_metrics_sam[group]["lenght"]),"+- {:.3f}".format(np.std(groups_metrics_sam[group]["pixel_acc"])))
        print("Mean IOU: {:.3f}".format(sum(groups_metrics_sam[group]["iou"]) / groups_metrics_sam[group]["lenght"]),"+- {:.3f}".format(np.std(groups_metrics_sam[group]["iou"])))
        print("Mean Dice: {:.3f}".format(sum(groups_metrics_sam[group]["dice"]) / groups_metrics_sam[group]["lenght"]),"+- {:.3f}".format(np.std(groups_metrics_sam[group]["dice"])))
        print("Number of images: ", groups_metrics_sam[group]["lenght"])
        print("Mean time: {:.3f}".format(sum(groups_metrics_sam[group]["time"]) / groups_metrics_sam[group]["lenght"]),"+- {:.3f}".format(np.std(groups_metrics_sam[group]["time"])))
    print()
    print("Groups metrics for Manual: ")
    for group in groups_metrics_manual.keys():
        print("Group: ", group)
        print("Mean pixel accuracy: {:.3f}".format(sum(groups_metrics_manual[group]["pixel_acc"]) / groups_metrics_manual[group]["lenght"]),"+- {:.3f}".format(np.std(groups_metrics_manual[group]["pixel_acc"])))
        print("Mean IOU: {:.3f}".format(sum(groups_metrics_manual[group]["iou"]) / groups_metrics_manual[group]["lenght"]),"+- {:.3f}".format(np.std(groups_metrics_manual[group]["iou"])))
        print("Mean Dice: {:.3f}".format(sum(groups_metrics_manual[group]["dice"]) / groups_metrics_manual[group]["lenght"]),"+- {:.3f}".format(np.std(groups_metrics_manual[group]["dice"])))
        print("Number of images: ", groups_metrics_manual[group]["lenght"])
        print("Mean time: {:.3f}".format(sum(groups_metrics_manual[group]["time"]) / groups_metrics_manual[group]["lenght"]),"+- {:.3f}".format(np.std(groups_metrics_manual[group]["time"])))
        print()



    print()
    print("Groups metrics for SAM + Manual: ")
    for group in groups_metrics_sam.keys():
        print("Group: ", group)
        print("Mean pixel accuracy: {:.3f}".format((sum(groups_metrics_sam[group]["pixel_acc"]) + sum(groups_metrics_manual[group]["pixel_acc"])) / (groups_metrics_sam[group]["lenght"] + groups_metrics_manual[group]["lenght"])),"+- {:.3f}".format(np.std(groups_metrics_sam[group]["pixel_acc"] + groups_metrics_manual[group]["pixel_acc"])))
        print("Mean IOU: {:.3f}".format((sum(groups_metrics_sam[group]["iou"]) + sum(groups_metrics_manual[group]["iou"])) / (groups_metrics_sam[group]["lenght"] + groups_metrics_manual[group]["lenght"])),"+- {:.3f}".format(np.std(groups_metrics_sam[group]["iou"] + groups_metrics_manual[group]["iou"])))
        print("Mean Dice: {:.3f}".format((sum(groups_metrics_sam[group]["dice"]) + sum(groups_metrics_manual[group]["dice"])) / (groups_metrics_sam[group]["lenght"] + groups_metrics_manual[group]["lenght"])),"+- {:.3f}".format(np.std(groups_metrics_sam[group]["dice"] + groups_metrics_manual[group]["dice"])))
        print("Number of images: ", groups_metrics_sam[group]["lenght"] + groups_metrics_manual[group]["lenght"])
        print("Mean time: {:.3f}".format((sum(groups_metrics_sam[group]["time"]) + sum(groups_metrics_manual[group]["time"])) / (groups_metrics_sam[group]["lenght"] + groups_metrics_manual[group]["lenght"])),"+- {:.3f}".format(np.std(groups_metrics_sam[group]["time"] + groups_metrics_manual[group]["time"])))
        print()

process_all_images(sessoes_folder, original_mask_folder)






   






   

Mean pixel accuracy for SAM: 0.907 +- 0.199
Mean IOU for SAM: 0.731 +- 0.240
Mean Dice for SAM: 0.814 +- 0.227
Number of images for SAM:  2100
Mean time for SAM: 594.104 +- 437.977

Mean pixel accuracy for Manual: 0.948 +- 0.045
Mean IOU for Manual: 0.745 +- 0.145
Mean Dice for Manual: 0.845 +- 0.110
Number of images for Manual:  2000
Mean time for Manual: 603.182 +- 300.440

Mean pixel accuracu for SAM + Manual: 0.927 +- 0.147
Mean IOU for SAM + Manual: 0.738 +- 0.199
Mean Dice for SAM + Manual: 0.829 +- 0.180
Number of images for SAM + Manual:  4100
Mean time for SAM + Manual: 598.532 +- 377.231
Groups metrics for SAM: 
Group:  group_1
Mean pixel accuracy: 0.950 +- 0.119
Mean IOU: 0.770 +- 0.200
Mean Dice: 0.850 +- 0.177
Number of images:  450
Mean time: 492.767 +- 162.041
Group:  group_2
Mean pixel accuracy: 0.831 +- 0.289
Mean IOU: 0.663 +- 0.299
Mean Dice: 0.745 +- 0.295
Number of images:  600
Mean time: 790.954 +- 685.349
Group:  group_3
Mean pixel accuracy: 0.939 +- 0.116
Mean I

In [11]:
group_name = "group_4"
folders_with_different_number_of_images = []

for sessao in os.listdir(sessoes_folder):
    if group_name in sessao:
        image_folder = os.path.join(sessoes_folder, sessao, "masked")
        num_images = len(os.listdir(image_folder))
        if num_images != 50:
            folders_with_different_number_of_images.append(sessao)

print(folders_with_different_number_of_images)

[]


In [1]:
import os
import cv2

original_mask_folder = 'pad_segmentation_all/all-mask'
sessoes_folder = "./sessoes"


def process_all_images(sessoes_folder, original_mask_folder):


    groups_metrics_sam = {"group_1":{}, "group_2":{}, "group_3":{}, "group_4":{}} 
    groups_metrics_manual = {"group_1":{}, "group_2":{}, "group_3":{}, "group_4":{}} 
    #create a dictionary to store the metrics for each group
    #create pixel accuracy, iou, dice and lenght for each group
    
    iou_sam_sum = 0
    iou_manual_sum = 0
    dice_sam_sum = 0
    dice_manual_sum = 0
    pixel_acc_sam_sum = 0
    pixel_acc_manual_sum = 0
    n_sam = 0
    n_manual = 0 

    for sessao in os.listdir(sessoes_folder):
        #get session group that is in the sessao name 
        for group in groups_metrics_sam.keys():
            if group in sessao:
                group_name = group
                break
        #get all images in the session folder 
        for image in os.listdir(os.path.join(sessoes_folder, sessao, "masked")):
            #get same mask from the original mask folder
            original_mask_name = image.replace('.jpg', '_segmentation.png')
            original_mask_path = os.path.join(original_mask_folder, original_mask_name)
            
            if not os.path.exists(original_mask_path):
                print(f"Image {original_mask_path} not found in the original mask folder")
                continue
   
            #load images
            mask = cv2.imread(os.path.join(sessoes_folder, sessao, "masked", image))
            original_mask_img = cv2.imread(original_mask_path)
            # Resize logic
            if original_mask_img.shape[:2] != mask.shape[:2]:
                target_shape = (min(original_mask_img.shape[1], mask.shape[1]), min(original_mask_img.shape[0], mask.shape[0]))
                original_mask_img = cv2.resize(original_mask_img, target_shape[::-1])  # Reverse shape for width, height
                mask = cv2.resize(mask, target_shape[::-1])

            original_mask = get_segmentation_mask_from_image(original_mask_img, original_mask_img.shape)
            mask = get_segmentation_mask_from_image(mask, original_mask_img.shape)

            if "sam" in sessao: 
                pixel_acc, iou, dice = calculate_metrics([mask], [original_mask])
                pixel_acc_sam_sum += pixel_acc
                iou_sam_sum += iou
                dice_sam_sum += dice
                n_sam += 1
                if group_name not in groups_metrics_sam.keys():
                    groups_metrics_sam[group_name] = {"pixel_acc":pixel_acc, "iou":iou, "dice":dice, "lenght":1}
                else:
                    groups_metrics_sam[group_name]["pixel_acc"] = groups_metrics_sam[group_name].get("pixel_acc", 0) + pixel_acc
                    groups_metrics_sam[group_name]["iou"] = groups_metrics_sam[group_name].get("iou", 0) + iou
                    groups_metrics_sam[group_name]["dice"] = groups_metrics_sam[group_name].get("dice", 0) + dice
                    groups_metrics_sam[group_name]["lenght"] = groups_metrics_sam[group_name].get("lenght", 0) + 1
            else:
                pixel_acc, iou, dice = calculate_metrics([mask], [original_mask])
                pixel_acc_manual_sum += pixel_acc
                iou_manual_sum += iou
                dice_manual_sum += dice
                n_manual += 1
                if group_name not in groups_metrics_manual.keys():
                    groups_metrics_manual[group_name] = {"pixel_acc":pixel_acc, "iou":iou, "dice":dice, "lenght":1}
                else:
                    groups_metrics_manual[group_name]["pixel_acc"] =  groups_metrics_manual[group_name].get("pixel_acc", 0) + pixel_acc
                    groups_metrics_manual[group_name]["iou"] = groups_metrics_manual[group_name].get("iou", 0) + iou
                    groups_metrics_manual[group_name]["dice"] = groups_metrics_manual[group_name].get("dice", 0) + dice
                    groups_metrics_manual[group_name]["lenght"] = groups_metrics_manual[group_name].get("lenght", 0) + 1


       


    print("Mean pixel accuracy for SAM: ", pixel_acc_sam_sum  / n_sam)
    print("Mean IOU for SAM: ", iou_sam_sum / n_sam)
    print("Mean Dice for SAM: ", dice_sam_sum / n_sam)
    print("Number of images for SAM: ", n_sam)
    print("Mean pixel accuracy for Manual: ", pixel_acc_manual_sum / n_manual)
    print("Mean IOU for Manual: ", iou_manual_sum / n_manual)
    print("Mean Dice for Manual: ", dice_manual_sum / n_manual)
    print("Number of images for Manual: ", n_manual)
    
    print("Groups metrics for SAM: ")
    
    for group in groups_metrics_sam.keys():
        print(group)
        print("Mean pixel accuracy: ", groups_metrics_sam[group]["pixel_acc"] / groups_metrics_sam[group]["lenght"])
        print("Mean IOU: ", groups_metrics_sam[group]["iou"] / groups_metrics_sam[group]["lenght"])
        print("Mean Dice: ", groups_metrics_sam[group]["dice"] / groups_metrics_sam[group]["lenght"])
        print("Number of images: ", groups_metrics_sam[group]["lenght"])
    print("Groups metrics for Manual: ")
    for group in groups_metrics_manual.keys():
        print(group)
        print("Mean pixel accuracy: ", groups_metrics_manual[group]["pixel_acc"] / groups_metrics_manual[group]["lenght"])
        print("Mean IOU: ", groups_metrics_manual[group]["iou"] / groups_metrics_manual[group]["lenght"])
        print("Mean Dice: ", groups_metrics_manual[group]["dice"] / groups_metrics_manual[group]["lenght"])
        print("Number of images: ", groups_metrics_manual[group]["lenght"])
import os
import cv2

original_mask_folder = 'pad_segmentation_all/all-mask'
sessoes_folder = "./sessoes"


def process_all_images(sessoes_folder, original_mask_folder):


    groups_metrics_sam = {"group_1":{}, "group_2":{}, "group_3":{}, "group_4":{}} 
    groups_metrics_manual = {"group_1":{}, "group_2":{}, "group_3":{}, "group_4":{}} 
    #create a dictionary to store the metrics for each group
    #create pixel accuracy, iou, dice and lenght for each group
    
    iou_sam_sum = 0
    iou_manual_sum = 0
    dice_sam_sum = 0
    dice_manual_sum = 0
    pixel_acc_sam_sum = 0
    pixel_acc_manual_sum = 0
    n_sam = 0
    n_manual = 0 

    for sessao in os.listdir(sessoes_folder):
        #get session group that is in the sessao name 
        for group in groups_metrics_sam.keys():
            if group in sessao:
                group_name = group
                break
        #get all images in the session folder 
        for image in os.listdir(os.path.join(sessoes_folder, sessao, "masked")):
            #get same mask from the original mask folder
            original_mask_name = image.replace('.jpg', '_segmentation.png')
            original_mask_path = os.path.join(original_mask_folder, original_mask_name)
            
            if not os.path.exists(original_mask_path):
                print(f"Image {original_mask_path} not found in the original mask folder")
                continue
   
            #load images
            mask = cv2.imread(os.path.join(sessoes_folder, sessao, "masked", image))
            original_mask_img = cv2.imread(original_mask_path)
            # Resize logic
            if original_mask_img.shape[:2] != mask.shape[:2]:
                target_shape = (min(original_mask_img.shape[1], mask.shape[1]), min(original_mask_img.shape[0], mask.shape[0]))
                original_mask_img = cv2.resize(original_mask_img, target_shape[::-1])  # Reverse shape for width, height
                mask = cv2.resize(mask, target_shape[::-1])

            original_mask = get_segmentation_mask_from_image(original_mask_img, original_mask_img.shape)
            mask = get_segmentation_mask_from_image(mask, original_mask_img.shape)

            if "sam" in sessao: 
                pixel_acc, iou, dice = calculate_metrics([mask], [original_mask])
                pixel_acc_sam_sum += pixel_acc
                iou_sam_sum += iou
                dice_sam_sum += dice
                n_sam += 1
                if group_name not in groups_metrics_sam.keys():
                    groups_metrics_sam[group_name] = {"pixel_acc":pixel_acc, "iou":iou, "dice":dice, "lenght":1}
                else:
                    groups_metrics_sam[group_name]["pixel_acc"] = groups_metrics_sam[group_name].get("pixel_acc", 0) + pixel_acc
                    groups_metrics_sam[group_name]["iou"] = groups_metrics_sam[group_name].get("iou", 0) + iou
                    groups_metrics_sam[group_name]["dice"] = groups_metrics_sam[group_name].get("dice", 0) + dice
                    groups_metrics_sam[group_name]["lenght"] = groups_metrics_sam[group_name].get("lenght", 0) + 1
            else:
                pixel_acc, iou, dice = calculate_metrics([mask], [original_mask])
                pixel_acc_manual_sum += pixel_acc
                iou_manual_sum += iou
                dice_manual_sum += dice
                n_manual += 1
                if group_name not in groups_metrics_manual.keys():
                    groups_metrics_manual[group_name] = {"pixel_acc":pixel_acc, "iou":iou, "dice":dice, "lenght":1}
                else:
                    groups_metrics_manual[group_name]["pixel_acc"] =  groups_metrics_manual[group_name].get("pixel_acc", 0) + pixel_acc
                    groups_metrics_manual[group_name]["iou"] = groups_metrics_manual[group_name].get("iou", 0) + iou
                    groups_metrics_manual[group_name]["dice"] = groups_metrics_manual[group_name].get("dice", 0) + dice
                    groups_metrics_manual[group_name]["lenght"] = groups_metrics_manual[group_name].get("lenght", 0) + 1


       


    print("Mean pixel accuracy for SAM: ", pixel_acc_sam_sum  / n_sam)
    print("Mean IOU for SAM: ", iou_sam_sum / n_sam)
    print("Mean Dice for SAM: ", dice_sam_sum / n_sam)
    print("Number of images for SAM: ", n_sam)
    print("Mean pixel accuracy for Manual: ", pixel_acc_manual_sum / n_manual)
    print("Mean IOU for Manual: ", iou_manual_sum / n_manual)
    print("Mean Dice for Manual: ", dice_manual_sum / n_manual)
    print("Number of images for Manual: ", n_manual)
    
    print("Groups metrics for SAM: ")
    
    for group in groups_metrics_sam.keys():
        print(group)
        print("Mean pixel accuracy: ", groups_metrics_sam[group]["pixel_acc"] / groups_metrics_sam[group]["lenght"])
        print("Mean IOU: ", groups_metrics_sam[group]["iou"] / groups_metrics_sam[group]["lenght"])
        print("Mean Dice: ", groups_metrics_sam[group]["dice"] / groups_metrics_sam[group]["lenght"])
        print("Number of images: ", groups_metrics_sam[group]["lenght"])
    print("Groups metrics for Manual: ")
    for group in groups_metrics_manual.keys():
        print(group)
        print("Mean pixel accuracy: ", groups_metrics_manual[group]["pixel_acc"] / groups_metrics_manual[group]["lenght"])
        print("Mean IOU: ", groups_metrics_manual[group]["iou"] / groups_metrics_manual[group]["lenght"])
        print("Mean Dice: ", groups_metrics_manual[group]["dice"] / groups_metrics_manual[group]["lenght"])
        print("Number of images: ", groups_metrics_manual[group]["lenght"])

process_all_images(sessoes_folder, original_mask_folder)






   






   

NameError: name 'get_segmentation_mask_from_image' is not defined

In [ ]:
process_all_images(sessoes_folder, original_mask_folder)
print("Metrics for all images")